In [ ]:
!pip install opencv-python-headless deepface

# Import necessary libraries
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Function to capture an image from the webcam
def capture_image():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await new Promise((resolve) => video.onloadedmetadata = resolve);
            video.play();

            // Resize video to match canvas dimensions
            const videoWidth = video.videoWidth;
            const videoHeight = video.videoHeight;
            canvas.width = videoWidth;
            canvas.height = videoHeight;

            // Capture frame on button click
            button.textContent = 'Capture';
            await new Promise((resolve) => {
                button.onclick = () => {
                    canvas.getContext('2d').drawImage(video, 0, 0, videoWidth, videoHeight);
                    stream.getTracks().forEach(track => track.stop());
                    resolve();
                };
            });

            const dataUrl = canvas.toDataURL('image/jpeg', 0.8);
            div.remove();
            return dataUrl;
        }
    ''')
    display(js)
    data = eval_js('takePhoto()')
    header, encoded = data.split(',', 1)
    image_data = b64decode(encoded)
    np_arr = np.frombuffer(image_data, dtype=np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img

# Function to analyze emotion using DeepFace
def analyze_emotion(img):
    from deepface import DeepFace
    result = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)
    return result

# Capture image from webcam
image = capture_image()

# Analyze emotion
analysis = analyze_emotion(image)

# Display the captured image and analysis
cv2_imshow(image)
print("Dominant Emotion:", analysis[0]['dominant_emotion'])
